In [2]:
import csv
import openpyxl
import os
import xlrd
from dateutil import parser
from datetime import datetime
from openpyxl.utils.datetime import from_excel as datetime_from_excel
import pandas as pd


################## CSV ##################

def how_many_csv_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0

    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        for row in csv.reader(file):
            for date_column in date_column_list:
                if date_column in row:
                    return num_rows_before_header, date_column
            num_rows_before_header += 1

    return None, None


def csv_to_pandas(filepath, date_column_list=['Compounds', 'Sampling Date']):
    num_rows_to_skip, date_column = how_many_csv_rows_to_skip(filepath, date_column_list)
    parsed_data = []

    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        reader = csv.reader(file)

        for _ in range(num_rows_to_skip):
            next(reader)
        
        headers = next(reader)
        date_column_index = headers.index(date_column)
        
        for row in reader:
            if all(item == '' for item in row):
                break
            row[date_column_index] = parser.parse(row[date_column_index])
            parsed_data.append(row)

    return pd.DataFrame(parsed_data, columns=headers)


################## XLSX ##################

def how_many_xlsx_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0
    wb = openpyxl.load_workbook(filepath, read_only=True)

    sheet = None
    for name in wb.sheetnames:
        if 'voc' in name.lower():
            sheet = wb[name]
            break
        if 'data' in name.lower() and 'metadata' not in name.lower():
            sheet = wb[name]
            break

    for row in sheet.iter_rows(values_only=True):
        for date_column in date_column_list:
            if date_column in row:
                wb.close()
                return num_rows_before_header, date_column
        num_rows_before_header += 1

    return None, None


def xlsx_to_pandas(filepath, date_column_list=['Compounds', 'Sampling Date']):
    num_rows_to_skip, date_column = how_many_xlsx_rows_to_skip(filepath, date_column_list)
    wb = openpyxl.load_workbook(filepath)

    sheet = None
    for name in wb.sheetnames:
        if 'voc' in name.lower():
            sheet = wb[name]
            break
        if 'data' in name.lower() and 'metadata' not in name.lower():
            sheet = wb[name]
            break

    for _ in range(num_rows_to_skip):
        next(sheet.iter_rows())

    header_row = next(sheet.iter_rows(min_row=sheet.min_row + num_rows_to_skip, max_row=sheet.min_row + num_rows_to_skip, values_only=True))
    date_column_index = header_row.index(date_column) if date_column in header_row else None

    parsed_data = []
    for row in sheet.iter_rows(min_row=sheet.min_row + num_rows_to_skip + 1, values_only=True):
        row = list(row)  # Convert the tuple to a list for modification

        date_cell = row[date_column_index]
        if isinstance(date_cell, datetime):
            pass
        elif isinstance(date_cell, float):
            row[date_column_index] = datetime_from_excel(date_cell)
        elif date_cell:
            row[date_column_index] = parser.parse(date_cell)
            
        parsed_data.append(row)

    return pd.DataFrame(parsed_data, columns=header_row)


################## XLS ##################

def get_xls_sheet(workbook):
    for name in workbook.sheet_names():
        if 'voc' in name.lower():
            return workbook[name]
        elif 'data' in name.lower():
            return workbook[name]
    return workbook.sheet_by_index(0)


def how_many_xls_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0
    wb = xlrd.open_workbook(filepath, encoding_override='ISO-8859-1')
    
    sheet = get_xls_sheet(wb)

    for row_idx in range(sheet.nrows):
        row = sheet.row_values(row_idx)
        for date_column in date_column_list:
            if date_column in row:
                return num_rows_before_header, date_column
        num_rows_before_header += 1

    return None, None


def xls_to_pandas(filepath, date_column_list=['Compounds', 'Sampling Date']):
    num_rows_to_skip, date_column = how_many_xls_rows_to_skip(filepath, date_column_list)
    wb = xlrd.open_workbook(filepath, encoding_override='ISO-8859-1')

    sheet = get_xls_sheet(wb)

    header_row = None
    for row_idx in range(num_rows_to_skip, sheet.nrows):
        row = sheet.row_values(row_idx)
        if date_column in row:
            header_row = row
            num_rows_to_skip = row_idx
            break

    date_column_index = header_row.index(date_column)

    parsed_data = []
    for row_idx in range(num_rows_to_skip + 1, sheet.nrows):
        row = sheet.row_values(row_idx)
        row[date_column_index] = xlrd.xldate_as_datetime(row[date_column_index], wb.datemode)
        parsed_data.append(row)

    return pd.DataFrame(parsed_data, columns=header_row)


In [3]:
################## CHECK IMPORT SCRIPTS ARE WORKING ##################

def check_all_imports(skip_xls=False, skip_xlsx=False, skip_csv=False, ignore_list=None):
    n_errors, n_files, n_xls, n_xlsx, n_csv, n_xls_complete, n_xlsx_complete, n_csv_complete  = 0, 0, 0, 0, 0, 0, 0, 0
    error_list = list()

    for root, dirs, files in os.walk('data'):
        for filename in files:
            filepath = os.path.join(root, filename)
            if filepath in ignore_list:
                continue

            extension = filepath.split('.')[-1].lower()
            if skip_xls and extension == 'xls':
                continue
            elif skip_xlsx and extension == 'xlsx':
                continue
            elif skip_csv and extension == 'csv':
                continue
            elif extension not in ['xls', 'xlsx', 'csv']:
                continue

            n_files += 1
            name = filepath.split('.')[0].lower()
            if name[-2:].lower() == 'fr':
                continue
            else:
                try:
                    if extension == 'xls':
                        n_xls += 1
                        xls_to_pandas(filepath)
                        n_xls_complete += 1
                    elif extension == 'xlsx':
                        n_xlsx += 1
                        xlsx_to_pandas(filepath)
                        n_xlsx_complete += 1
                    elif extension == 'csv':
                        n_csv += 1
                        csv_to_pandas(filepath)
                        n_csv_complete += 1
                except:
                    n_errors += 1
                    error_list.append(filepath)

    print(f'{n_errors} errors / {n_files} total files')
    print(f'xls: {n_xls_complete} out of {n_xls}')
    print(f'csv: {n_csv_complete} out of {n_csv}')
    print(f'xlsx: {n_xlsx_complete} out of {n_xlsx}')

    return error_list

In [4]:
ignore_list = [
    'data\\ddmmyyyy.xlsx', 'data\\mmddyyyy.xlsx', 'data\\yyyyddmm.xlsx', 'data\\yyyymmdd.xlsx',
    'data\\ddmmyyyy.csv', 'data\\mmddyyyy.csv', 'data\\yyyyddmm.csv', 'data\\yyyymmdd.csv',
    'data\\2006\\S62601_VOCS.csv', 'data\\2007\\S62601_VOCS.csv',    # sideways csv for some reason
    'data\\2008\\S90227_VOC.csv', 'data\\2009\\S90227_VOC.csv', 'data\\2010\\S90227_VOC.csv',  # sampling data relocated
    'data\\2015\\ChangeLog_Jan2017.xls'
]

##### XLSX works (or seems to)

In [5]:
xlsx_errors = check_all_imports(skip_xls=True, skip_xlsx=False, skip_csv=True, ignore_list=ignore_list)

0 errors / 236 total files
xls: 0 out of 0
csv: 0 out of 0
xlsx: 118 out of 118


##### CSV works (finally)

In [6]:
csv_errors = check_all_imports(skip_xls=True, skip_xlsx=True, skip_csv=False, ignore_list=ignore_list)

0 errors / 786 total files
xls: 0 out of 0
csv: 628 out of 628
xlsx: 0 out of 0


##### XLS works too!

In [7]:
xls_errors = check_all_imports(skip_xls=False, skip_xlsx=True, skip_csv=True, ignore_list=ignore_list)

0 errors / 260 total files
xls: 220 out of 220
csv: 0 out of 0
xlsx: 0 out of 0


##### Check Errors

In [8]:
# check errors to see what's up
first_n = 10
xls_errors[:first_n]

# to copy paste into markdown for link followage
for link in xls_errors[:first_n]:
    file = link.split('\\')[-1]
    print(f'- [{file}]({link})')

##### Confirm everything works correctly with random inspection

In [20]:
list_of_files = list()
for root, dirs, files in os.walk('data'):
    for filename in files:
        filepath = os.path.join(root, filename)
        if filepath not in ignore_list:
            list_of_files.append(filepath)

list_of_files = [x for x in list_of_files if x not in ignore_list]
list_of_files = [x for x in list_of_files if x.split('.')[-2][-2:].lower() != 'fr']

In [39]:
import random

file = random.choice(list_of_files)
print(file)
extension = file.split('.')[-1].lower()

df = None
if extension == 'xls':
    df = xls_to_pandas(file)
elif extension == 'xlsx':
    df = xlsx_to_pandas(file)
elif extension == 'csv':
    df = csv_to_pandas(file)
else:
    print('NO EXTENSION')

if isinstance(df, pd.DataFrame):
    display(df.head())
    display(df.dtypes)

data\2009\S50115_VOC.csv


,Compounds,Ethane,Ethylene,Acetylene,Propylene,Propane,1-Propyne,Isobutane,1-Butene/Isobutene,"1,3-Butadiene",...,Hexanal,"2,5-Dimethylbenzaldehyde",,Sample ID#,Sample Date,Canister ID#,Sample Volume,NAPS ID,START TIME,DURATION
0,2009-01-01,3.325301612,1.378226347,0.804072519,0.497,2.2227,0.0698,1.692,0.4109,0.0868,...,0.069444444,0,,mt66ao.d,1/1/09,EPS 492,500,50115,00:00,24
1,2009-01-07,4.476014909,3.569755858,3.73162836,1.7577,3.9235,0.2163,2.369,1.0594,0.2964,...,0.159722222,0,,mt67ao.d,1/7/09,EPS 404,500,50115,00:00,24
2,2009-01-13,4.429111885,2.915607904,2.443188267,0.8956,3.6059,0.1535,3.2517,0.7302,0.1708,...,0.180555556,0,,mt76ao.d,1/13/09,WTC 022,500,50115,00:00,24
3,2009-01-19,3.714540047,3.79823256,2.691056266,1.4535,3.5773,0.218,2.9744,1.038,0.2908,...,0.180555556,0,,mt77ao.d,1/19/09,EPS 187,500,50115,00:00,24
4,2009-01-25,4.244258305,2.19325981,1.76161822,0.897,3.3157,0.1487,1.8255,0.5583,0.1704,...,,,,mt87ao.d,1/25/09,EPS 039,500,50115,00:00,24


Compounds        datetime64[ns]
Ethane                   object
Ethylene                 object
Acetylene                object
Propylene                object
                      ...      
Canister ID#             object
Sample Volume            object
NAPS ID                  object
START TIME               object
DURATION                 object
Length: 185, dtype: object

#### EDGE CASE: these csv use non-standard encoding (obviously)

    With default encoding:
        79 errors / 786 total files
        xls: 0 out of 0
        csv: 549 out of 628
        xlsx: 0 out of 0
    with encoding='ISO-8859-1':
        0 errors / 786 total files
        xls: 0 out of 0
        csv: 628 out of 628
        xlsx: 0 out of 0

#### EDGE CASE: some csv have metadata at the bottom - need to break once a dead row begins

These all have metadata after the rows of data, need to break after a row of all blanks
- [S010102_VOC_2017_EN.csv](data\2017\S010102_VOC_2017_EN.csv)
- [S030118_VOC_2017_EN.csv](data\2017\S030118_VOC_2017_EN.csv)
- [S040203_VOC_2017_EN.csv](data\2017\S040203_VOC_2017_EN.csv)
- [S040208_VOC_2017_EN.csv](data\2017\S040208_VOC_2017_EN.csv)
- [S040901_VOC_2017_EN.csv](data\2017\S040901_VOC_2017_EN.csv)
- [S050103_VOC_2017_EN.csv](data\2017\S050103_VOC_2017_EN.csv)
- [S050122_VOC_2017_EN.csv](data\2017\S050122_VOC_2017_EN.csv)
- [S050129_VOC_2017_EN.csv](data\2017\S050129_VOC_2017_EN.csv)
- [S050133_VOC_2017_EN.csv](data\2017\S050133_VOC_2017_EN.csv)
- [S050134_VOC_2017_EN.csv](data\2017\S050134_VOC_2017_EN.csv)
- [S050136_VOC_2017_EN.csv](data\2017\S050136_VOC_2017_EN.csv)
- [S050308_VOC_2017_EN.csv](data\2017\S050308_VOC_2017_EN.csv)
- [S060104_VOC_2017_EN.csv](data\2017\S060104_VOC_2017_EN.csv)
- [S060211_VOC_2017_EN.csv](data\2017\S060211_VOC_2017_EN.csv)
- [S060435_VOC_2017_EN.csv](data\2017\S060435_VOC_2017_EN.csv)
- [S060438_VOC_2017_EN.csv](data\2017\S060438_VOC_2017_EN.csv)
- [S060440_VOC_2017_EN.csv](data\2017\S060440_VOC_2017_EN.csv)
- [S060512_VOC_2017_EN.csv](data\2017\S060512_VOC_2017_EN.csv)
- [S060904_VOC_2017_EN.csv](data\2017\S060904_VOC_2017_EN.csv)
- [S061007_VOC_2017_EN.csv](data\2017\S061007_VOC_2017_EN.csv)

#### EDGE CASE: some csv have "Sampling Date" (not "Compound") and/or need cropping

In [ ]:
# this file needs to use 'Sampling Date' and to crop useless columns
link = r'data\2014\S060512_VOC_2014.csv'
print(link)
print()
csv_to_pandas(link, ['Sampling Date']).replace('', np.nan).dropna(axis=1)

data\2014\S060512_VOC_2014.csv



,Sampling Date,NAPS ID,Ethane (ug/m3),Ethylene (ug/m3),Acetylene (ug/m3),Propylene (ug/m3),Propane (ug/m3),1-Propyne (ug/m3),Isobutane (ug/m3),1-Butene/Isobutene (ug/m3),...,Chlorobenzene (ug/m3),Benzylchloride (ug/m3),Bromoform (ug/m3),"1,4-Dichlorobutane (ug/m3)","1,1,2,2-Tetrachloroethane (ug/m3)","1,3-Dichlorobenzene (ug/m3)","1,4-Dichlorobenzene (ug/m3)","1,2-Dichlorobenzene (ug/m3)","1,2,4-Trichlorobenzene (ug/m3)",Hexachlorobutadiene (ug/m3)
0,2014-01-05,060512,7.215,2.093,1.056,0.320,3.645,,1.679,0.259,...,0.000,0.000,0.017,,0.000,0.001,0.037,0.002,0.003,0.002
1,2014-01-11,060512,11.176,2.463,1.313,0.574,6.727,,3.488,0.522,...,0.014,0.000,0.019,,0.000,0.001,0.162,0.002,0.004,0.002
2,2014-01-17,060512,6.651,1.295,0.813,0.309,4.343,,1.190,0.214,...,0.009,0.000,0.017,,0.000,0.001,0.041,0.002,0.003,0.002
3,2014-01-23,060512,4.172,1.444,0.794,0.347,2.496,,1.268,0.224,...,0.008,0.000,0.024,,0.000,0.002,0.024,0.003,0.006,0.000
4,2014-01-29,060512,4.822,1.155,0.749,0.273,3.750,,0.840,0.167,...,0.007,0.000,0.020,,0.000,0.001,0.024,0.002,0.004,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2014-12-07,060512,4.265,0.786,0.464,0.134,2.530,,0.862,0.129,...,0.008,0.000,0.027,,-999.000,0.002,0.050,0.002,0.004,0.004
57,2014-12-13,060512,10.427,1.451,0.640,0.317,8.170,,1.738,0.164,...,0.008,0.000,0.016,,0.000,0.003,0.045,0.005,0.009,0.006
58,2014-12-19,060512,4.341,0.853,0.466,0.207,2.488,,0.720,0.183,...,0.008,0.000,0.018,,0.000,0.002,0.034,0.004,0.006,0.006
59,2014-12-25,060512,5.959,0.666,0.473,0.161,4.690,,1.037,0.104,...,0.011,0.000,0.018,,0.000,0.002,0.041,0.003,0.005,0.006
